In [1]:
import numpy as np
import pandas as pd
import math
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [2]:
data=pd.read_csv('./data/training_set_2.csv',index_col=0)

In [3]:
data

,A,R,N,D,C,E,Q,G,H,I,...,EMS_1b_features_25,EMS_1b_features_26,EMS_1b_features_27,EMS_1b_features_28,EMS_1b_features_29,EMS_1b_features_30,EMS_1b_features_31,EMS_1b_features_32,EMS_1b_features_33,label
ENSCINP00000007207.3,0.070500,0.077300,0.067000,0.068200,0.007950,0.059100,0.055700,0.036400,0.026100,0.056800,...,-7.938638,-10.105667,-8.311391,-8.116248,-1.371832,-1.694018,-1.732382,-1.672009,-18.805689,1
ENSTRUP00000057393.1,0.086022,0.071685,0.039427,0.057348,0.032258,0.039427,0.064516,0.064516,0.021505,0.035842,...,-5.904070,-9.656778,-7.462701,-7.683102,-1.319787,-1.620215,-1.550143,-1.556692,-18.451927,1
ENSDARP00000132131.1,0.135000,0.000000,0.020800,0.015600,0.000000,0.052100,0.031300,0.078100,0.000000,0.005210,...,-6.942954,-10.369159,-9.145744,-8.375875,-1.630719,-1.664754,-1.974382,-1.864807,-19.627354,1
ENSGALP00000024462.4,0.093500,0.077900,0.028000,0.028000,0.009350,0.049800,0.034300,0.071700,0.028000,0.024900,...,-5.713566,-9.351521,-6.372884,-6.733988,-1.071575,-1.393639,-1.307287,-1.316862,-17.450712,1
ENSGALP00000064711.2,0.080000,0.045714,0.028571,0.057143,0.040000,0.068571,0.034286,0.062857,0.011429,0.045714,...,-4.805295,-8.205428,-6.192973,-6.212621,-0.979434,-1.392809,-1.260624,-1.294684,-17.986265,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4ERNA,0.076125,0.069204,0.041522,0.038062,0.010381,0.076125,0.062284,0.048443,0.020761,0.051903,...,-4.694638,-8.387127,-5.825796,-5.744020,-0.708508,-1.335728,-1.119150,-1.096704,-17.683250,0
4JRNA,0.083333,0.053763,0.029570,0.053763,0.016129,0.075269,0.048387,0.064516,0.016129,0.043011,...,-7.818323,-10.525990,-8.220621,-8.460024,-1.580368,-1.724772,-1.831649,-1.805930,-18.067270,0
3HRNA,0.046875,0.125000,0.000000,0.062500,0.000000,0.125000,0.015625,0.078125,0.015625,0.078125,...,-7.789245,-9.932685,-8.125821,-7.930046,-1.470230,-1.716438,-1.572652,-1.692108,-18.297180,0
3JRNA,0.056818,0.056818,0.039773,0.068182,0.011364,0.073864,0.028409,0.039773,0.028409,0.056818,...,-6.486643,-9.022049,-7.289336,-7.181287,-1.349755,-1.585473,-1.560659,-1.553373,-18.763400,0


In [4]:
data=data.sample(frac=1,random_state=1)
train,test=train_test_split(data,test_size=0.2,random_state=1)
train_x=train.drop(["label"],axis="columns")
train_y=train["label"]
test_x=test.drop(["label"],axis="columns")
test_y=test["label"]

In [5]:
train_x

,A,R,N,D,C,E,Q,G,H,I,...,EMS_1b_features_24,EMS_1b_features_25,EMS_1b_features_26,EMS_1b_features_27,EMS_1b_features_28,EMS_1b_features_29,EMS_1b_features_30,EMS_1b_features_31,EMS_1b_features_32,EMS_1b_features_33
4J20A,0.181818,0.011364,0.034091,0.045455,0.022727,0.000000,0.034091,0.136364,0.011364,0.022727,...,-4.423692,-4.433928,-9.240186,-6.652843,-6.979736,-0.950299,-1.491869,-1.520078,-1.383216,-18.433340
1MTPA,0.154799,0.095975,0.012384,0.061920,0.003096,0.046440,0.012384,0.052632,0.034056,0.018576,...,-1.540192,-7.451056,-10.832590,-8.228182,-8.471477,-1.494302,-1.805757,-1.740003,-1.859344,-19.316730
ENSSSCP00000047640.1,0.036200,0.045200,0.058800,0.036200,0.040700,0.067900,0.013600,0.054300,0.036200,0.063300,...,-1.205201,-7.489838,-9.607475,-8.194365,-8.189975,-1.692650,-1.672207,-1.808206,-1.806987,-17.375536
ENSCINP00000036114.1,0.090300,0.039700,0.043300,0.046900,0.007220,0.046900,0.043300,0.068600,0.036100,0.083000,...,-2.305725,-6.840336,-10.063556,-7.842552,-8.132273,-1.472136,-1.646860,-1.731705,-1.736634,-18.149561
4G6TB,0.146341,0.073171,0.024390,0.036585,0.000000,0.048780,0.097561,0.097561,0.012195,0.060976,...,-2.850491,-8.061125,-10.881370,-8.932117,-8.758325,-1.730847,-1.858907,-1.913313,-1.970303,-18.680050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3JV1A,0.060440,0.054945,0.038462,0.087912,0.005495,0.104396,0.043956,0.065934,0.021978,0.043956,...,-1.812482,-8.175171,-10.529090,-8.353060,-8.400045,-1.462166,-1.758100,-1.734277,-1.766733,-18.954160
ENSCHIP00000010520.1,0.078125,0.171875,0.031250,0.015625,0.046875,0.046875,0.031250,0.000000,0.000000,0.031250,...,-1.122967,-8.036904,-10.166411,-8.555175,-8.595782,-1.822590,-1.800910,-1.932916,-1.927080,-17.382648
ENSOARP00000019679.1,0.054054,0.103604,0.018018,0.054054,0.018018,0.040541,0.049550,0.054054,0.049550,0.040541,...,-1.134844,-6.416158,-9.401396,-7.963693,-7.814702,-1.606975,-1.679765,-1.667665,-1.793477,-17.557421
2XOCA,0.068966,0.068966,0.038314,0.053640,0.080460,0.049808,0.065134,0.057471,0.022989,0.015326,...,0.307905,-6.243873,-9.806445,-6.493751,-7.018152,-1.061173,-1.473970,-1.340405,-1.433316,-18.493700


In [6]:
cv_params = {'n_estimators': [100,200,300,400, 500, 600, 700, 800]}
other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1,
                    'silent':True, 'objective':'multi:softmax','num_class':2}

model =  xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train_x, train_y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[20:09:03] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=0, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_bin=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=5,
                                     max_leaves=None, min_child_weight=1,
                                     missing=nan, monotone_constraints=None,
               

In [7]:
optimized_GBM.best_params_

{'n_estimators': 500}

In [8]:
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

参数的最佳取值：{'n_estimators': 500}
最佳模型得分:0.9303904923599321


In [9]:
cv_params = {'n_estimators': [425,450,475,500,525,550,575]}
other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1,
                    'silent':True, 'objective':'multi:softmax','num_class':2}

model =  xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train_x, train_y)
means=optimized_GBM.cv_results_["mean_test_score"]
params=optimized_GBM.cv_results_['params']
# for mean,param in zip(means,params):
#     print('%f with %r'%(mean,param))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[02:38:06] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


参数的最佳取值：{'n_estimators': 550}
最佳模型得分:0.9305036785512166


In [10]:
n_estimators=optimized_GBM.best_params_["n_estimators"]

In [11]:
#2.Tuning:min_child_weight and max_depth：
cv_params = {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_child_weight': [1, 2, 3, 4, 5, 6]}
other_params = {'learning_rate': 0.1, 'n_estimators': n_estimators, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1,
                    'silent':True, 'objective':'multi:softmax','num_class':2}

model =  xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train_x, train_y)
means=optimized_GBM.cv_results_["mean_test_score"]
params=optimized_GBM.cv_results_['params']
# for mean,param in zip(means,params):
#     print('%f with %r'%(mean,param))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[15:13:00] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


参数的最佳取值：{'max_depth': 5, 'min_child_weight': 6}
最佳模型得分:0.9322014714204867


In [12]:
max_depth=optimized_GBM.best_params_["max_depth"]
min_child_weight=optimized_GBM.best_params_["min_child_weight"]

In [13]:
#3.Tuning:gamma：
cv_params = {'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]}
other_params = {'learning_rate': 0.1, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_child_weight': min_child_weight, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1,
                    'silent':True, 'objective':'multi:softmax','num_class':2}

model =  xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train_x, train_y)
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[21:24:44] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


参数的最佳取值：{'gamma': 0.1}
最佳模型得分:0.9324278438030561


In [14]:
#4.Tuning:subsample and colsample_bytree：
cv_params = {'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]}
gamma=optimized_GBM.best_params_["gamma"]
other_params = {'learning_rate': 0.1, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_child_weight': min_child_weight, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': gamma, 'reg_alpha': 0, 'reg_lambda': 1,
                    'silent':True, 'objective':'multi:softmax','num_class':2}

model =  xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train_x, train_y)
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[08:08:58] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


参数的最佳取值：{'colsample_bytree': 0.8, 'subsample': 0.8}
最佳模型得分:0.9324278438030561


In [15]:
#5.Tuning:reg_alpha and reg_lambda：
cv_params = {'reg_alpha': [0.05, 0,0.1, 1, 2, 3], 'reg_lambda': [0.05,0, 0.1, 1, 2, 3]}
subsample=optimized_GBM.best_params_["subsample"]
colsample_bytree=optimized_GBM.best_params_["colsample_bytree"]
other_params = {'learning_rate': 0.1, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_child_weight': min_child_weight, 'seed': 0,
                    'subsample': subsample, 'colsample_bytree': colsample_bytree, 'gamma': gamma, 'reg_alpha': 0, 'reg_lambda': 1,
                    'silent':True, 'objective':'multi:softmax','num_class':2}

model =  xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train_x, train_y)
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[12:39:27] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


参数的最佳取值：{'reg_alpha': 0, 'reg_lambda': 1}
最佳模型得分:0.9324278438030561


In [16]:
#6.Tuning:learning_rate：
cv_params = {'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]}
reg_alpha=optimized_GBM.best_params_["reg_alpha"]
reg_lambda=optimized_GBM.best_params_["reg_lambda"]
other_params = {'learning_rate': 0.1, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_child_weight': min_child_weight,'seed': 0,
                    'subsample': subsample, 'colsample_bytree': colsample_bytree, 'gamma': gamma, 'reg_alpha': reg_alpha, 'reg_lambda': reg_lambda,
                    'silent':True, 'objective':'multi:softmax','num_class':2}

model =  xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='accuracy', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(train_x, train_y)
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
means=optimized_GBM.cv_results_["mean_test_score"]
params=optimized_GBM.cv_results_['params']
for mean,param in zip(means,params):
    print('%f with %r'%(mean,param))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[17:15:19] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


参数的最佳取值：{'learning_rate': 0.1}
最佳模型得分:0.9324278438030561
0.922581 with {'learning_rate': 0.01}
0.930843 with {'learning_rate': 0.05}
0.930504 with {'learning_rate': 0.07}
0.932428 with {'learning_rate': 0.1}
0.929032 with {'learning_rate': 0.2}


In [17]:
import pickle

In [18]:
# save model to file 模型保存
pickle.dump(model, open("./XGBoost/model.pickle.dat", "wb"))
 
# load model from file 模型加载
# loaded_model = pickle.load(open("pima.pickle.dat", "rb"))